# Create and Train Model

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import config
from data_loading import load_data_from_dir


# 1 Two Models to Train Each Batch : Wav Features

## 1.1 Load Split Data

### 1.1.1 Load Training Data

In [4]:
train_wav_dirs = [config.TRAIN_WAV_MFCCS, config.TRAIN_AUG_WAV_MFCCS]
#train_lms_dirs = [config.TRAIN_LMS_MFCCS, config.TRAIN_AUG_LMS_MFCCS]

X_train = []
y_train = []

for train_dir in train_wav_dirs:
    for file_name in os.listdir(train_dir):
        if file_name.endswith('.npy'):
            file_path = os.path.join(train_dir, file_name)
            mfccs = np.load(file_path)
            X_train.append(mfccs)
            label = file_name.split('.npy')[0]
            y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [5]:
X_train.shape, y_train.shape

((1716, 40, 775), (1716,))

### 1.1.2 Load Test Data

In [6]:
test_dirs = [config.TEST_WAV_MFCCS]
#test_dirs = [config.TEST_LMS_MFCCS]

X_test = []
y_test = []

for test_dir in test_dirs:
    for file_name in os.listdir(test_dir):
        if file_name.endswith('.npy'):
            file_path = os.path.join(test_dir, file_name)
            mfccs = np.load(file_path)
            X_test.append(mfccs)
            label = file_name.split('.npy')[0]
            y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)


In [7]:
X_test.shape, y_test.shape

((394, 40, 775), (394,))

### 1.1.3 Load Validation Data

In [8]:
val_dirs = [config.VAL_WAV_MFCCS]
#val_dirs = [config.VAL_LMS_MFCCS]

X_val = []
y_val = []

for val_dir in val_dirs:
    for file_name in os.listdir(val_dir):
        if file_name.endswith('.npy'):
            file_path = os.path.join(val_dir, file_name)
            mfccs = np.load(file_path)
            X_val.append(mfccs)
            label = file_name.split('.npy')[0]
            y_val.append(label)

X_val = np.array(X_val)
y_val = np.array(y_val)


In [9]:
X_val.shape, y_val.shape

((344, 40, 775), (344,))

Get number of classes from filename for model

In [10]:
filenames, labels = load_data_from_dir(config.WAV_DIR_PATH)

emotion_labels = [filename.split('-')[2] for filename in filenames] # based on filename identifiers https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio

num_classes = len(np.unique(emotion_labels))

print("Number of classes (emotions):", num_classes)

Number of classes (emotions): 8


## 1.2 Define Model

In [11]:
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

model = models.Sequential([
    layers.Input(shape=X_train_reshaped.shape[1:]),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


## 1.3 Compile the Model

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 1.3.1 Need to Encode the Labels as they are String Data

In [13]:
emotions = [label.split('-')[2] for label in y_train]

label_encoder = LabelEncoder()
emotions_encoded = label_encoder.fit_transform(emotions)
emotions_encoded = emotions_encoded.reshape(-1, 1)

encoder = OneHotEncoder(categories='auto', sparse_output=False) # try sparse true 
y_train_encoded = encoder.fit_transform(emotions_encoded)

y_test_encoded = encoder.transform(label_encoder.transform([label.split('-')[2] for label in y_test]).reshape(-1, 1))
y_val_encoded = encoder.transform(label_encoder.transform([label.split('-')[2] for label in y_val]).reshape(-1, 1))


In [14]:
print(np.unique(emotions))

['01' '02' '03' '04' '05' '06' '07' '08']


## 1.4 Train the Model

In [15]:
X_val_reshaped = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

history = model.fit(X_train_reshaped, y_train_encoded,
                    epochs=10, 
                    batch_size=32,
                    validation_data=(X_val_reshaped, y_val_encoded)) 


Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 47s 808ms/step - accuracy: 0.1530 - loss: 8.6515 - val_accuracy: 0.3401 - val_loss: 1.7976
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 749ms/step - accuracy: 0.3450 - loss: 1.7306 - val_accuracy: 0.3372 - val_loss: 1.6960
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 767ms/step - accuracy: 0.4372 - loss: 1.5098 - val_accuracy: 0.4477 - val_loss: 1.5346
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 43s 788ms/step - accuracy: 0.5207 - loss: 1.3155 - val_accuracy: 0.4564 - val_loss: 1.4593
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 43s 800ms/step - accuracy: 0.5988 - loss: 1.1123 - val_accuracy: 0.4767 - val_loss: 1.3957
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 39s 711ms/step - accuracy: 0.6688 - loss: 0.8913 - val_accuracy: 0.5581 - val_loss: 1.2262
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 40s 742ms/step - accuracy: 0.7596 - loss: 0.6815 - val_accuracy: 0.6337 - val_loss: 1.0927
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 42s 777ms/step - accuracy: 0.8098 - loss: 0.5253 - val_accu

* dropout : https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/
* evaluate: https://medium.com/pydatascience/4-techniques-of-evaluating-the-performance-of-deep-learning-models-using-validation-35ef4b12e8d8
* batch normalisation : https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/
* global average poolong: https://medium.com/aaweg-i-nterview/computer-vision-global-average-pooling-d2bbdfbd713f
* regularising : https://medium.com/intelligentmachines/convolutional-neural-network-and-regularization-techniques-with-tensorflow-and-keras-5a09e6e65dc7

## 1.5 Evaluate Model

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded)
print('Test accuracy:', test_accuracy)

# 2 Two Models to Train Each Batch : LMS Features

## 2.1 Load Split Data

### 2.1.1 Load Training Data

In [30]:
#train_wav_dirs = [config.TRAIN_WAV_MFCCS, config.TRAIN_AUG_WAV_MFCCS]
train_lms_dirs = [config.TRAIN_LMS_MFCCS, config.TRAIN_AUG_LMS_MFCCS]

X_train = []
y_train = []

for train_dir in train_lms_dirs:
    for file_name in os.listdir(train_dir):
        if file_name.endswith('.npy'):
            file_path = os.path.join(train_dir, file_name)
            mfccs = np.load(file_path)
            X_train.append(mfccs)
            label = file_name.split('.npy')[0]
            y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [31]:
X_train.shape, y_train.shape

((1717, 40, 775), (1717,))

### 2.1.2 Load Testing Data

In [32]:
#test_dirs = [config.TEST_WAV_MFCCS]
test_dirs = [config.TEST_LMS_MFCCS]

X_test = []
y_test = []

for test_dir in test_dirs:
    for file_name in os.listdir(test_dir):
        if file_name.endswith('.npy'):
            file_path = os.path.join(test_dir, file_name)
            mfccs = np.load(file_path)
            X_test.append(mfccs)
            label = file_name.split('.npy')[0]
            y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [33]:
X_test.shape, y_test.shape

((216, 40, 775), (216,))

### 2.1.3 Load Validation Data

In [34]:
#val_dirs = [config.VAL_WAV_MFCCS]
val_dirs = [config.VAL_LMS_MFCCS]

X_val = []
y_val = []

for val_dir in val_dirs:
    for file_name in os.listdir(val_dir):
        if file_name.endswith('.npy'):
            file_path = os.path.join(val_dir, file_name)
            mfccs = np.load(file_path)
            X_val.append(mfccs)
            label = file_name.split('.npy')[0]
            y_val.append(label)

X_val = np.array(X_val)
y_val = np.array(y_val)

In [35]:
X_val.shape, y_val.shape

((184, 40, 775), (184,))

## 2.2 Define Model

In [50]:
# needs a lot of tweaking for lms features ? See previous list of techniques to prevent overfitting

X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

model = models.Sequential([
    layers.Input(shape=X_train_reshaped.shape[1:]),
    #layers.Conv2D(32, (5, 5), activation='relu'),
    layers.Conv2D(32, (3, 3), activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.GlobalAveragePooling2D(),
    #layers.flatten(),
    layers.Dense(128, activation='relu'),
    #layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [51]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
emotions = [label.split('-')[2] for label in y_train]

label_encoder = LabelEncoder()
emotions_encoded = label_encoder.fit_transform(emotions)
emotions_encoded = emotions_encoded.reshape(-1, 1)

encoder = OneHotEncoder(categories='auto', sparse_output=False) # try sparse true
y_train_encoded = encoder.fit_transform(emotions_encoded)

y_test_encoded = encoder.transform(label_encoder.transform([label.split('-')[2] for label in y_test]).reshape(-1, 1))
y_val_encoded = encoder.transform(label_encoder.transform([label.split('-')[2] for label in y_val]).reshape(-1, 1))


## 2.3 Train Model

In [53]:
X_val_reshaped = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

history = model.fit(X_train_reshaped, y_train_encoded,
                    epochs=20, 
                    batch_size=32,
                    validation_data=(X_val_reshaped, y_val_encoded)) 


Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 36s 622ms/step - accuracy: 0.1307 - loss: 3.1494 - val_accuracy: 0.1141 - val_loss: 2.0760
Epoch 2/20
26/54 ━━━━━━━━━━━━━━━━━━━━ 17s 618ms/step - accuracy: 0.1447 - loss: 2.0706

KeyboardInterrupt: 

* the validation data is not too far off, increase epochs. If no change, convergance, tweak model
* incxreased epoch led to dropped val accuracy around 10, suggesting overfitting
 
* evaluation as follows after 10 epichs
 * 7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.3556 - loss: 1.7614
 * Test accuracy: 0.3611111044883728

## 2.4 Evaluate Model

In [42]:
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_encoded)
print('Test accuracy:', test_accuracy)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.3556 - loss: 1.7614
Test accuracy: 0.3611111044883728
